# [Amazon Personalize](https://aws.amazon.com/personalize/) introduces - Trending Now

This notebook will walk you through an example of using Trending Now recipe in [Amazon Personalize](https://aws.amazon.com/personalize/)

User interests can change based on a variety of factors, such as external events or the interests of other users. It is critical for websites and apps to tailor their recommendations to these changing interests to improve user engagement. With [Trending-Now](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-trending-now.html), you can surface items from your catalogue that are rising in popularity faster with higher velocity than other items, such as trending news, popular social content or newly released movies. Amazon Personalize looks for items that are rising in popularity at a faster rate than other catalogue items to help users discover items that are engaging their peers. Amazon Personalize also allows customers to define the time periods over which trends are calculated depending on their unique business context, with options for every 30 mins, 1 hour, 3 hours or 1 day, based on the most recent interactions data from users.
This notebook will demonstrate how the new recipe aws-trending-now (or aws-vod-trending-now for recommenders) can help recommend the top trending items from the interactions dataset.  

The estimated time to run through this notebook is about 60 minutes.  

## How to use the Notebook

The code is broken up into cells like the one below. There's a triangular Run button at the top of this page that you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` while in the cell to execute it and move onto the next one.

As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.

Simply follow the instructions below and execute the cells to get started.

## Introduction to Amazon Personalize

[Amazon Personalize](https://aws.amazon.com/personalize/) is a fully managed machine learning service that makes it easy for developers to deliver personalized experiences to their users. It enables you to improve customer engagement by powering personalized product and content recommendations in websites, applications, and targeted marketing campaigns. You can get started without any prior machine learning (ML) experience, using APIs to easily build sophisticated personalization capabilities in a few clicks. All your data is encrypted to be private and secure, and is only used to create recommendations for your users. 

You can start using Amazon Personalize with a simple three step process, which only takes a few clicks in the AWS console, or a set of simple API calls. 

First, point Amazon Personalize to user data, catalog data, and activity stream of views, clicks, purchases, etc. in Amazon S3 or upload using a simple API call. 

Second, with a single click in the console or an API call, train a private recommendation model for your data. 

Third, retrieve personalized recommendations for any user by creating a recommender, and using the GetRecommendations API.

If you are not familiar with Amazon Personalize, you can learn more about the service on by looking at [Github Sample Notebooks](https://github.com/aws-samples/amazon-personalize-samples) and [Product Documentation](https://docs.aws.amazon.com/personalize/latest/dg/what-is-personalize.html).

## Imports
Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK for python) and [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/) which are core data science tools.

In [ ]:
# Get the latest version of botocore to ensure we have the latest features in the SDK
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade --no-deps --force-reinstall botocore

In [ ]:
# Imports
import boto3
import json as json
import numpy as np
import pandas as pd
import time
import datetime
import uuid

## Specify an S3 Bucket and Data Output Location

Amazon Personalize will need an S3 bucket to act as the source of your data. The code bellow will create a bucket with a unique `bucket_name`.

The Amazon S3 bucket needs to be in the same region as the Amazon Personalize resources. 

In [ ]:
# Sets the same region as current Amazon SageMaker Notebook
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# Or you can specify the region where your bucket and model will be domiciled
# region = "us-east-1" 

s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalize-trending-now"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket = bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
except s3.exceptions.BucketAlreadyOwnedByYou:
    print("Bucket already exists. Using bucket", bucket_name)

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.Session(region_name=region).client('personalize')
personalize_events = boto3.Session(region_name=region).client('personalize-events')
personalize_runtime = boto3.Session(region_name=region).client('personalize-runtime')

## Download, Prepare, and Upload Training Data

For this notebook walkthrough, we will use Movielens public dataset, available at https://grouplens.org/datasets/movielens/. Follow the link to learn more about the data and potential uses.

First we need to download the data (training data). In this tutorial, for the interactions data we will be using ratings history from the movies review dataset, MovieLens. The dataset contains the user_id, rating, item_id, the interactions between the users and items and the time this interaction took place (timestamp which is given as unix epoch time). The dataset also contains movie title information to map the movie id to the actual title and genres. 

### Download and Explore the Interactions and Items Dataset

In [ ]:
data_dir = "blog_data"
!mkdir $data_dir

In [ ]:
!cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!cd $data_dir && unzip ml-latest-small.zip
dataset_dir = data_dir + "/ml-latest-small/"

The dataset has been successfully downloaded 

Lets learn more about the dataset by viewing its charateristics

In [ ]:
!pygmentize $dataset_dir/README.txt

From the README, we see there is a file ratings.csv that should work as a proxy for our interactions data, after all rating a film definitely is a form of interacting with it. The dataset also has some genre information as some movie genome data. In this POC we will focus on the interactions data.

In [ ]:
ratings_df = pd.read_csv(dataset_dir + '/ratings.csv')

In [ ]:
ratings_df.info()

In [ ]:
ratings_df.head(10)

In [ ]:
ratings_df.describe()

Now lets create titles dataframe with the movie titles information which we will use for post processing the recommendations output for better readability

In [ ]:
titles_df = pd.read_csv(dataset_dir + '/movies.csv')

In [ ]:
titles_df.info()

In [ ]:
titles_df.head()

In [ ]:
titles_df.describe()

Renaming the columns to perform the JOIN operations in the later steps of this notebook

In [ ]:
titles_df = titles_df.rename(columns = {'movieId':'ITEM_ID', 'title':'TITLE', 'genres':'GENRES'})

In [ ]:
titles_df.head()

## Prepare the Interactions Data

### Convert the ratings data to Click and Watch event interactions

Since this is a dataset of an explicit feedback movie ratings, it includes movies rated from 1 to 5. We want to include only moves that were "liked" by the users, and simulate a dataset of data that would be gathered by a VOD platform. In order to do that, we will filter out all interactions under 2 out of 5, and create two event types: "Click" and and "Watch". We will then assign all movies rated 2 and above as "Click" and movies rated 4 and above as both "Click" and "Watch".

Note that for a real data set you would actually model based on implicit feedback such as clicks, watches and/or explicit feedback such as ratings, likes etc.

In [ ]:
watched_df = ratings_df.copy()
watched_df = watched_df[watched_df['rating'] > 3]
watched_df = watched_df[['userId', 'movieId', 'timestamp']]
watched_df['EVENT_TYPE']='Watch'
watched_df.head()

In [ ]:
clicked_df = ratings_df.copy()
clicked_df = clicked_df[clicked_df['rating'] > 1]
clicked_df = clicked_df[['userId', 'movieId', 'timestamp']]
clicked_df['EVENT_TYPE']='Click'
clicked_df.head()

In [ ]:
interactions_df = clicked_df.copy()
interactions_df = interactions_df.append(watched_df)
interactions_df.sort_values("timestamp", axis = 0, ascending = True, 
                 inplace = True, na_position ='last') 

In [ ]:
interactions_df.info()

In [ ]:
interactions_df.head(10)

In [ ]:
interactions_df.describe()

In [ ]:
interactions_df = interactions_df.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 'timestamp':'TIMESTAMP'})

In [ ]:
interactions_df.head()

In [ ]:
interactions_file_path = 'curated_interactions_training_data.csv'

In the cell below, we will write our cleaned data to a file named "final_training_data.csv

In [ ]:
interactions_df.to_csv(interactions_file_path)

## Configure an S3 bucket and an IAM role

So far, we have downloaded, manipulated, and saved the data onto the Amazon EBS instance attached to instance running this Jupyter notebook. However, Amazon Personalize will need an S3 bucket to act as the source of your data, as well as IAM roles for accessing that bucket. Let's set all of that up.


## Set the S3 bucket policy
Amazon Personalize needs to be able to read the contents of your S3 bucket. So add a bucket policy which allows that.

Note: Make sure the role you are using to run the code in this notebook has the necessary permissions to modify the S3 bucket policy.

In [ ]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

### Upload Interactions data to S3
Now that our training data is ready for Amazon Personalize,the next step is to upload it to the s3 bucket created earlier

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_file_path).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_file_path
    

## Create and Wait for Dataset Group
The largest grouping in Personalize is a Dataset Group, this will isolate your data, event trackers, solutions, Recommenders, and campaigns. Grouping things together that share a common collection of data. Feel free to alter the name below if you'd like. 

When you create a Domain dataset group, you choose your domain. The domain you specify determines the default schemas for datasets and the use cases that are available for recommenders. 

You can find more information about creating a Domain dataset group in [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/create-domain-dataset-group.html).

### Create Dataset Group

In [ ]:
response = personalize.create_dataset_group(
    name='trending_now_dataset_group'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

Wait for Dataset Group to Have ACTIVE Status
Before we can use the Dataset Group in any items below it must be active, execute the cell below and wait for it to show active.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Create Interactions Schema
A core component of how Personalize understands your data comes from the Schema that is defined below. This configuration tells the service how to digest the data provided via your CSV file. Note the columns and types align to what was in the file you created above.

In [ ]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "EVENT_TYPE", # "Watch", "Click", etc.
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }  
    ],
    "version": "1.0"
}


create_schema_response = personalize.create_schema(
    name = "trending-now-interaction-dataset-schema",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

## Create Datasets
After the group, the next thing to create is the datasets where your data will be uploaded to in Amazon Personalize.

### Create Interactions Dataset

In [ ]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "trending-now-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Create Personalize Role
Also Amazon Personalize needs the ability to assume Roles in AWS in order to have the permissions to execute certain tasks, the lines below grant that.

Note: Make sure the role you are using to run the code in this notebook has the necessary permissions to create a role.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleTrendingNow"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)


## Import the data
Earlier you created the DatasetGroup and Dataset to house your information, now you will execute an import job that will load the data from S3 into Amazon Personalize for usage building your model.
### Create Interactions Dataset Import Job

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "trending_now_interactions_import",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_file_path)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

### Wait for Dataset Import Jobs to Have ACTIVE Status
It can take a while before the import jobs complete, please wait until you see that they are active below.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours

while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("InteractionsDatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Training the Trending-Now model - Solution & Solution Version
With the interactions dataset imported into dataset group, we will next create solution and solution version using the trending-now recipe. First, let's initialize the trending-now-recipe for creating the custom solution. Please note the new featureTransformationParameter that we are setting for this recipe while creating the solution. Its value can be selected from the set of [30 minutes, 1 hour, 3 hours, 1 day].

This input parameter should be decided based on the data distribution for each input interactions dataset. If the use case falls in video domain, where the trending movies or TV series change every 1 day, then we should select 1 day (or) if the use case falls in news domain, then we should select 30 minute so that fast changing news stories are showcased as trending items in recommendations.


In [ ]:
trending_now_recipe = "arn:aws:personalize:::recipe/aws-trending-now"
solution_response = personalize.create_solution(name="trending-now-solution", 
                                                recipeArn=trending_now_recipe, 
                                                datasetGroupArn=dataset_group_arn,
                                                solutionConfig = {
                                                    "featureTransformationParameters": {
                                                        'trend_discovery_frequency': '30 minutes'
                                                    }
                                                }   
                                               )
solution_arn = solution_response['solutionArn']

In [ ]:
solution_version_response = personalize.create_solution_version(solutionArn=solution_arn, 
                                                         trainingMode="FULL")
solution_version_arn = solution_version_response["solutionVersionArn"]
print(json.dumps(solution_version_response, indent=4, default=str))

In [ ]:
in_progress_solution_versions = [solution_version_arn]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("Solution Version is still IN_PROGRESS state")
        
    time.sleep(60)

In [ ]:
personalize.describe_solution_version(solutionVersionArn=solution_version_arn)

## Campaign
With solution and solution version created, we will now create a campaign to get recommendations. Please note, with trending now recipe, we can use both [filter](https://docs.aws.amazon.com/personalize/latest/dg/filter.html) and [promotions](https://docs.aws.amazon.com/personalize/latest/dg/promoting-items.html) while calling getRecommendations over an active campaign (or recommender).

In [ ]:
campaign_response = personalize.create_campaign(
    name="trending-now-campaign",
    solutionVersionArn=solution_version_arn,
    minProvisionedTPS=1,
)
campaign_arn = campaign_response['campaignArn']

In [ ]:
in_progress_campaigns = [campaign_arn]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        describe_campaign_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = describe_campaign_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("Campaign is still IN_PROGRESS state")
        
    time.sleep(60)

## Getting recommendations
Now that the campaign has been trained, lets have a look at the recommendations we can get for our users!

In [ ]:
item_ids = personalize_runtime.get_recommendations(campaignArn=campaign_arn, 
                                                   numResults=25)

The results from the GetRecommendations call includes the id’s of recommended items. Below is the code to map the id’s to the actual movie titles for readability.

In [ ]:
items = []
for item in item_ids["itemList"]:
    items.append(int(item["itemId"]))
recommended_items_df = pd.DataFrame (items, columns = ['ITEM_ID'])

In [ ]:
recommendations = recommended_items_df.merge(titles_df, on="ITEM_ID")
recommendations = recommendations[["ITEM_ID", "TITLE"]]
recommendations.head(10)

## EventTracker to ingest Put Events to trigger auto-updates
To use this recipe effectively and to catch trendiness for the 'Trend discovery frequency' configured - one should use putEvents API to continuously ingest new events into the system. Trending now model will incorporate these new events with automatic model updates (at no cost to end customer) to show latest trending now items.
Lets create an event tracker to ingest new events into our models.

In [ ]:
event_tracker_response = personalize.create_event_tracker(
    name='trending-now-event-tracker',
    datasetGroupArn=dataset_group_arn
)
tracking_id = event_tracker_response['trackingId']
event_tracker_response
time.sleep(60)

### Prepare latest interactions  
Let us create some dummy latest interaction events with some random user_ids and item_ids to push it to the Amaon Personalize using event tracker. We will use the interaction timestamp to be of the current timestamp to replicate latest real time ingestions. 

In [ ]:
user_rand = interactions_df['USER_ID'].sample(10)
items_rand = interactions_df['ITEM_ID'].sample(10)

In [ ]:
list(user_rand)

Above are the randomly selected USER_ID’s that are used for generating incremental interactions.

In [ ]:
list(items_rand)

Above are the randomly selected ITEM_ID’s that are used for generating incremental interactions.

In [ ]:
titles_df[titles_df['ITEM_ID'].isin(list(items_rand))]

In [ ]:
for i in range(0,25):
    for user_id in user_rand:
        session_id = str(uuid.uuid4())
        for item_id in items_rand:
            time.sleep(1)
            time_epoch = int(time.time())
            response = personalize_events.put_events(
                trackingId=tracking_id,
                userId=str(user_id),
                sessionId=session_id,
                eventList=[
                    {
                        'eventType': 'Watch',
                        'itemId': str(item_id),
                        'sentAt': time_epoch
                    }
                ]
            )
            print ("Iteration: %s, User: %s, Item: %s, EpochTime: %s" %(i, user_id, item_id, str(time_epoch)))

Approximately it takes 40+ minutes to push all the dummy interactions.

After you push the incremental interactions dataset, wait for Trend Discovery Frequency time configured plus additional processing time (15 minutes) for the new recommendations to get reflected. 

## Getting trending-now recommendations
Now that the latest interactions are pushed to Personalize, lets have a look at the latest trending now recommendations we can get for our users!

In [ ]:
item_ids = personalize_runtime.get_recommendations(campaignArn=campaign_arn, 
                                                   numResults=25)

In [ ]:
items = []
for item in item_ids["itemList"]:
    items.append(int(item["itemId"]))
recommended_items_df = pd.DataFrame (items, columns = ['ITEM_ID'])

In [ ]:
recommendations = recommended_items_df.merge(titles_df, on="ITEM_ID")
recommendations = recommendations[["ITEM_ID", "TITLE"]]
recommendations.head(30)

## Review
The above GetRecommendations call includes the id’s of recommended items. Now we see the ITEM_ID’s recommended are from the incremental interactions dataset that we had provided to the Personalize model. Amazon Personalize trending now recipe looks for items that are rising in popularity at a faster rate in the latest interactions that are streaming in and recommend them as trending now items.
Using the notebook code above you have successfully trained a trending now model to generate item recommendations  that are rapidly becoming popular with your users and tailor your recommendations for the fast changing trends in the user interest. 
Going forward, you can adapt this code to create other recommenders.

## Cleanup Resources 
This section contains instructions on how to clean up the resources created in this notebook

In [ ]:
print ('dataset_group_arn:', dataset_group_arn)
print ('role_name:', role_name)
print ('region:', region)

In [ ]:
import sys
import getopt
import logging
import botocore
import boto3
import time
from packaging import version
from time import sleep
from botocore.exceptions import ClientError

logger = logging.getLogger()
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
logger.setLevel(logging.INFO)
logger.addHandler(handler)

personalize = None

In [ ]:
def _get_dataset_group_arn(dataset_group_name):
    dsg_arn = None

    paginator = personalize.get_paginator('list_dataset_groups')
    for paginate_result in paginator.paginate():
        for dataset_group in paginate_result["datasetGroups"]:
            if dataset_group['name'] == dataset_group_name:
                dsg_arn = dataset_group['datasetGroupArn']
                break

        if dsg_arn:
            break

    if not dsg_arn:
        raise NameError(f'Dataset Group "{dataset_group_name}" does not exist; verify region is correct')

    return dsg_arn

def _get_solutions(dataset_group_arn):
    solution_arns = []

    paginator = personalize.get_paginator('list_solutions')
    for paginate_result in paginator.paginate(datasetGroupArn = dataset_group_arn):
        for solution in paginate_result['solutions']:
            solution_arns.append(solution['solutionArn'])

    return solution_arns

def _delete_campaigns(solution_arns):
    campaign_arns = []

    for solution_arn in solution_arns:
        paginator = personalize.get_paginator('list_campaigns')
        for paginate_result in paginator.paginate(solutionArn = solution_arn):
            for campaign in paginate_result['campaigns']:
                if campaign['status'] in ['ACTIVE', 'CREATE FAILED']:
                    logger.info('Deleting campaign: ' + campaign['campaignArn'])

                    personalize.delete_campaign(campaignArn = campaign['campaignArn'])
                elif campaign['status'].startswith('DELETE'):
                    logger.warning('Campaign {} is already being deleted so will wait for delete to complete'.format(campaign['campaignArn']))
                else:
                    raise Exception('Campaign {} has a status of {} so cannot be deleted'.format(campaign['campaignArn'], campaign['status']))

                campaign_arns.append(campaign['campaignArn'])

    max_time = time.time() + 30*60 # 30 mins
    while time.time() < max_time:
        for campaign_arn in campaign_arns:
            try:
                describe_response = personalize.describe_campaign(campaignArn = campaign_arn)
                logger.debug('Campaign {} status is {}'.format(campaign_arn, describe_response['campaign']['status']))
            except ClientError as e:
                error_code = e.response['Error']['Code']
                if error_code == 'ResourceNotFoundException':
                    campaign_arns.remove(campaign_arn)

        if len(campaign_arns) == 0:
            logger.info('All campaigns have been deleted or none exist for dataset group')
            break
        else:
            logger.info('Waiting for {} campaign(s) to be deleted'.format(len(campaign_arns)))
            time.sleep(20)

    if len(campaign_arns) > 0:
        raise Exception('Timed out waiting for all campaigns to be deleted')

def _delete_solutions(solution_arns):
    for solution_arn in solution_arns:
        try:
            describe_response = personalize.describe_solution(solutionArn = solution_arn)
            solution = describe_response['solution']
            if solution['status'] in ['ACTIVE', 'CREATE FAILED']:
                logger.info('Deleting solution: ' + solution_arn)

                personalize.delete_solution(solutionArn = solution_arn)
            elif solution['status'].startswith('DELETE'):
                logger.warning('Solution {} is already being deleted so will wait for delete to complete'.format(solution_arn))
            else:
                raise Exception('Solution {} has a status of {} so cannot be deleted'.format(solution_arn, solution['status']))
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code != 'ResourceNotFoundException':
                raise e

    max_time = time.time() + 30*60 # 30 mins
    while time.time() < max_time:
        for solution_arn in solution_arns:
            try:
                describe_response = personalize.describe_solution(solutionArn = solution_arn)
                logger.debug('Solution {} status is {}'.format(solution_arn, describe_response['solution']['status']))
            except ClientError as e:
                error_code = e.response['Error']['Code']
                if error_code == 'ResourceNotFoundException':
                    solution_arns.remove(solution_arn)

        if len(solution_arns) == 0:
            logger.info('All solutions have been deleted or none exist for dataset group')
            break
        else:
            logger.info('Waiting for {} solution(s) to be deleted'.format(len(solution_arns)))
            time.sleep(20)

    if len(solution_arns) > 0:
        raise Exception('Timed out waiting for all solutions to be deleted')

def _delete_event_trackers(dataset_group_arn):
    event_tracker_arns = []

    event_trackers_paginator = personalize.get_paginator('list_event_trackers')
    for event_tracker_page in event_trackers_paginator.paginate(datasetGroupArn = dataset_group_arn):
        for event_tracker in event_tracker_page['eventTrackers']:
            if event_tracker['status'] in [ 'ACTIVE', 'CREATE FAILED' ]:
                logger.info('Deleting event tracker {}'.format(event_tracker['eventTrackerArn']))
                personalize.delete_event_tracker(eventTrackerArn = event_tracker['eventTrackerArn'])
            elif event_tracker['status'].startswith('DELETE'):
                logger.warning('Event tracker {} is already being deleted so will wait for delete to complete'.format(event_tracker['eventTrackerArn']))
            else:
                raise Exception('Solution {} has a status of {} so cannot be deleted'.format(event_tracker['eventTrackerArn'], event_tracker['status']))

            event_tracker_arns.append(event_tracker['eventTrackerArn'])

    max_time = time.time() + 30*60 # 30 mins
    while time.time() < max_time:
        for event_tracker_arn in event_tracker_arns:
            try:
                describe_response = personalize.describe_event_tracker(eventTrackerArn = event_tracker_arn)
                logger.debug('Event tracker {} status is {}'.format(event_tracker_arn, describe_response['eventTracker']['status']))
            except ClientError as e:
                error_code = e.response['Error']['Code']
                if error_code == 'ResourceNotFoundException':
                    event_tracker_arns.remove(event_tracker_arn)

        if len(event_tracker_arns) == 0:
            logger.info('All event trackers have been deleted or none exist for dataset group')
            break
        else:
            logger.info('Waiting for {} event tracker(s) to be deleted'.format(len(event_tracker_arns)))
            time.sleep(20)

    if len(event_tracker_arns) > 0:
        raise Exception('Timed out waiting for all event trackers to be deleted')

def _delete_filters(dataset_group_arn):
    filter_arns = []

    filters_response = personalize.list_filters(datasetGroupArn = dataset_group_arn, maxResults = 100)
    for filter in filters_response['Filters']:
        logger.info('Deleting filter ' + filter['filterArn'])
        personalize.delete_filter(filterArn = filter['filterArn'])
        filter_arns.append(filter['filterArn'])

    max_time = time.time() + 30*60 # 30 mins
    while time.time() < max_time:
        for filter_arn in filter_arns:
            try:
                describe_response = personalize.describe_filter(filterArn = filter_arn)
                logger.debug('Filter {} status is {}'.format(filter_arn, describe_response['filter']['status']))
            except ClientError as e:
                error_code = e.response['Error']['Code']
                if error_code == 'ResourceNotFoundException':
                    filter_arns.remove(filter_arn)

        if len(filter_arns) == 0:
            logger.info('All filters have been deleted or none exist for dataset group')
            break
        else:
            logger.info('Waiting for {} filter(s) to be deleted'.format(len(filter_arns)))
            time.sleep(20)

    if len(filter_arns) > 0:
        raise Exception('Timed out waiting for all filter to be deleted')

def _delete_datasets_and_schemas(dataset_group_arn):
    dataset_arns = []
    schema_arns = []

    dataset_paginator = personalize.get_paginator('list_datasets')
    for dataset_page in dataset_paginator.paginate(datasetGroupArn = dataset_group_arn):
        for dataset in dataset_page['datasets']:
            describe_response = personalize.describe_dataset(datasetArn = dataset['datasetArn'])
            schema_arns.append(describe_response['dataset']['schemaArn'])

            if dataset['status'] in ['ACTIVE', 'CREATE FAILED']:
                logger.info('Deleting dataset ' + dataset['datasetArn'])
                personalize.delete_dataset(datasetArn = dataset['datasetArn'])
            elif dataset['status'].startswith('DELETE'):
                logger.warning('Dataset {} is already being deleted so will wait for delete to complete'.format(dataset['datasetArn']))
            else:
                raise Exception('Dataset {} has a status of {} so cannot be deleted'.format(dataset['datasetArn'], dataset['status']))

            dataset_arns.append(dataset['datasetArn'])

    max_time = time.time() + 30*60 # 30 mins
    while time.time() < max_time:
        for dataset_arn in dataset_arns:
            try:
                describe_response = personalize.describe_dataset(datasetArn = dataset_arn)
                logger.debug('Dataset {} status is {}'.format(dataset_arn, describe_response['dataset']['status']))
            except ClientError as e:
                error_code = e.response['Error']['Code']
                if error_code == 'ResourceNotFoundException':
                    dataset_arns.remove(dataset_arn)

        if len(dataset_arns) == 0:
            logger.info('All datasets have been deleted or none exist for dataset group')
            break
        else:
            logger.info('Waiting for {} dataset(s) to be deleted'.format(len(dataset_arns)))
            time.sleep(20)

    if len(dataset_arns) > 0:
        raise Exception('Timed out waiting for all datasets to be deleted')

    for schema_arn in schema_arns:
        try:
            logger.info('Deleting schema ' + schema_arn)
            personalize.delete_schema(schemaArn = schema_arn)
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == 'ResourceInUseException':
                logger.info('Schema {} is still in-use by another dataset (likely in another dataset group)'.format(schema_arn))
            else:
                raise e

    logger.info('All schemas used exclusively by datasets have been deleted or none exist for dataset group')

def _delete_dataset_group(dataset_group_arn):
    logger.info('Deleting dataset group ' + dataset_group_arn)
    personalize.delete_dataset_group(datasetGroupArn = dataset_group_arn)

    max_time = time.time() + 30*60 # 30 mins
    while time.time() < max_time:
        try:
            describe_response = personalize.describe_dataset_group(datasetGroupArn = dataset_group_arn)
            logger.debug('Dataset group {} status is {}'.format(dataset_group_arn, describe_response['datasetGroup']['status']))
            break
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == 'ResourceNotFoundException':
                logger.info('Dataset group {} has been fully deleted'.format(dataset_group_arn))
            else:
                raise e

        logger.info('Waiting for dataset group to be deleted')
        time.sleep(20)

def delete_dataset_groups(dataset_group_arns, region = None):
    global personalize
    personalize = boto3.client(service_name = 'personalize', region_name = region)

    for dataset_group_arn in dataset_group_arns:
        logger.info('Dataset Group ARN: ' + dataset_group_arn)

        solution_arns = _get_solutions(dataset_group_arn)

        # 1. Delete campaigns
        _delete_campaigns(solution_arns)

        # 2. Delete solutions
        _delete_solutions(solution_arns)

        # 3. Delete event trackers
        _delete_event_trackers(dataset_group_arn)

        # 4. Delete filters
        _delete_filters(dataset_group_arn)

        # 5. Delete datasets and their schemas
        _delete_datasets_and_schemas(dataset_group_arn)

        # 6. Delete dataset group
        _delete_dataset_group(dataset_group_arn)

        logger.info(f'Dataset group {dataset_group_arn} fully deleted')

In [ ]:
delete_dataset_groups([dataset_group_arn], region)

In [ ]:
iam = boto3.client('iam')

In [ ]:
iam.list_attached_role_policies(
    RoleName = role_name
)

In [ ]:
# detaching the role policies
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

In [ ]:
iam.delete_role(
    RoleName = role_name
)

In [ ]:
!rm -rf $data_dir

## Deleting the S3 bucket
To delete an S3 bucket, it first needs to be empty. The easiest way to delete an S3 bucket, is just to navigate to S3 in the AWS console, delete the objects in the bucket, and then delete the S3 bucket itself.